In [32]:
using JuMP, Gurobi, CSV, DataFrames

### Data loading

In [33]:
Blue1 = Matrix(CSV.read("processed_data/Blue1_dataset.csv", DataFrame))[:,2:end];
Blue2 = Matrix(CSV.read("processed_data/Blue2_dataset.csv", DataFrame))[:,2:end];

Orange1 = Matrix(CSV.read("processed_data/Orange1_dataset.csv", DataFrame))[:,2:end];
Orange2 = Matrix(CSV.read("processed_data/Orange2_dataset.csv", DataFrame))[:,2:end];

Green1 = Matrix(CSV.read("processed_data/Green1_dataset.csv", DataFrame))[:,2:end];
Green2 = Matrix(CSV.read("processed_data/Green2_dataset.csv", DataFrame))[:,2:end];

Red1 = Matrix(CSV.read("processed_data/Red1_dataset.csv", DataFrame))[:,2:end];
Red2 = Matrix(CSV.read("processed_data/Red2_dataset.csv", DataFrame))[:,2:end];

In [34]:
AvgFlowBlue = cat(Blue1, Blue2, dims = 3);
AvgFlowOrange = cat(Orange1, Orange2, dims = 3);
AvgFlowGreen = cat(Green1, Green2, dims = 3);
AvgFlowRed = cat(Red1, Red2, dims = 3);

In [57]:
cost_blue = 10000;
cost_orange = 9000;
cost_green = 8000;
cost_red = 10000;

capacity_blue = 1000;
capacity_orange = 1100;
capacity_green = 900;
capacity_red = 1100;

q = 2.4;

## Blue model

In [58]:
stations_b, times, directions = size(AvgFlowBlue);

In [59]:
modelBlue = Model(Gurobi.Optimizer)

@variable(modelBlue, x[1:directions, 1:times] >= 0)
@variable(modelBlue, u[1:directions, 1:times, 1:stations_b] >= 0)

@constraint(modelBlue, [d=1:directions, j=1:times, i=1:stations_b], 
    u[d,j,i] + capacity_blue * x[d,j] >= AvgFlowBlue[i,j,d])

@objective(modelBlue, Min, sum(sum(cost_blue * x[d,j] + 
            sum(q * u[d,j,i] for i=1:stations_b) for d=1:directions) for j=1:times))

optimize!(modelBlue)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-04
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 216 rows, 234 columns and 432 nonzeros
Model fingerprint: 0x60fdc5c7
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e+00, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 1e+04]
Presolve removed 18 rows and 18 columns
Presolve time: 0.00s
Presolved: 198 rows, 216 columns, 396 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.170884e+04   0.000000e+00      0s
      90    5.3523600e+05   0.000000e+00   0.000000e+00      0s

Solved in 90 iterations and 0.00 seconds (0.00 work units)
Optimal objective  5.352360000e+05

User-callback calls 131, time in user-callback 0.00 sec


## Orange model

In [62]:
stations_o, times, directions = size(AvgFlowOrange);

In [63]:
modelOrange = Model(Gurobi.Optimizer)

@variable(modelOrange, x[1:directions, 1:times] >= 0)
@variable(modelOrange, u[1:directions, 1:times, 1:stations_o] >= 0)

@constraint(modelOrange, [d=1:directions, j=1:times, i=1:stations_o], 
    u[d,j,i] + capacity_orange * x[d,j] >= AvgFlowOrange[i,j,d])

@objective(modelOrange, Min, sum(sum(cost_orange * x[d,j] + 
            sum(q * u[d,j,i] for i=1:stations_o) for d=1:directions) for j=1:times))

optimize!(modelOrange)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-04
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 360 rows, 378 columns and 720 nonzeros
Model fingerprint: 0xae1c1785
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e+00, 9e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 18 rows and 18 columns
Presolve time: 0.00s
Presolved: 342 rows, 360 columns, 684 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.506750e+04   0.000000e+00      0s
      72    8.0325109e+05   0.000000e+00   0.000000e+00      0s

Solved in 72 iterations and 0.00 seconds (0.00 work units)
Optimal objective  8.032510909e+05

User-callback calls 113, time in user-callback 0.00 sec


## Red model

In [88]:
stations_r, times, directions = size(AvgFlowRed);
lines = 2;

In [89]:
red_stations = Matrix(CSV.read("processed_data/Red1_dataset.csv", DataFrame))[:,1];

Red1 = ["Alewife", "Davis", "Porter", "Harvard", "Central", "Kendall/MIT", "Charles/MGH",
     "Park Street", "Downtown Crossing", "South Station", "Broadway", "Andrew", "JFK/Umass",
      "Savin Hill", "Fields Corner", "Shawmut", "Ashmont"];

Red2 = ["Alewife", "Davis", "Porter", "Harvard", "Central", "Kendall/MIT", "Charles/MGH",
     "Park Street", "Downtown Crossing", "South Station", "Broadway", "Andrew", "JFK/Umass",
      "North Quincy", "Wollaston", "Quincy Center", "Quincy Adams", "Braintree"];

In [90]:
z_red = zeros((2,22));

for i = 1:22
    if red_stations[i] in Red1
        z_red[1,i] = 1
    end
    if red_stations[i] in Red2
        z_red[2,i] = 1
    end
end

In [95]:
modelRed = Model(Gurobi.Optimizer)

@variable(modelRed, x[1:directions, 1:times, 1:lines] >= 0)
@variable(modelRed, u[1:directions, 1:times, 1:stations_r] >= 0)

@constraint(modelRed, [d=1:directions, j=1:times, i=1:stations_r], 
    u[d,j,i] + (sum(capacity_red * x[d,j,l] * z_red[l,i] for l=1:lines)) >= AvgFlowRed[i,j,d])

@objective(modelRed, Min, sum( sum(cost_red * x[d,j,l] for l=1:lines ) + 
                sum(q * u[d,j,i] for i=1:stations_r) for d=1:directions,  j=1:times))

optimize!(modelRed)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-04
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 396 rows, 432 columns and 1026 nonzeros
Model fingerprint: 0xddbf7f99
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e+00, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+04]
Presolve removed 28 rows and 28 columns
Presolve time: 0.00s
Presolved: 368 rows, 404 columns, 961 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.862347e+04   0.000000e+00      0s
      93    1.1677925e+06   0.000000e+00   0.000000e+00      0s

Solved in 93 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.167792509e+06

User-callback calls 134, time in user-callback 0.00 sec


### Green line

In [114]:
stations_g, times, directions = size(AvgFlowGreen);
lines = 4;

In [115]:
green_stations = Matrix(CSV.read("processed_data/Green1_dataset.csv", DataFrame))[:,1];

Green1 = ["Lechmere", "Science Park", "North Station", "Haymarket", "Government Center", "Park Street", "Boylston",
 "Arlington", "Copley", "Hynes Convention Center", "Kenmore", "Blandford Street", "Boston Univ. East", 
 "Boston Univ. Central", "Boston Univ. West", "Saint Paul Street", "Pleasant Street", "Babcock Street",
 "Harvard Ave.","Griggs Street", "Allston Street", "Warren Street", "Washington Street", "Sutherland Road",
"Chiswick Road", "Chestnut Hill Ave.", "South Street"];

Green2 = ["Government Center", "Park Street", "Boylston", "Arlington", "Copley", "Hynes Convention Center", "Kenmore", 
"Saint Mary Street", "Hawes Street", "Kent Street", "Saint Paul Street", "Coolidge Corner", "Summit Ave.", 
"Brandon Hall", "Fairbanks Street", "Washington Square", "Tappan Street", "Dean Road", "Englewood Ave.", "Cleveland Circle"];

Green3 = ["Lechmere", "Science Park", "North Station", "Haymarket", "Government Center", "Park Street", "Boylston", 
"Arlington", "Copley", "Hynes Convention Center", "Kenmore", "Fenway", "Longwood", "Brookline Village", "Brookline Hills",
"Beaconsfield", "Reservoir", "Chestnut Hill Ave.", "Newton Centre", "Newton Highlands", "Eliot", "Waban", "Woodland", 
"Riverside"];

Green4 = ["Lechmere", "Science Park", "North Station", "Haymarket", "Government Center", "Park Street", "Boylston",
    "Arlington", "Copley", "Prudential", "Symphony", "Northeastern University", "Museum of Fine Arts", "Longwood",
    "Brigham Circle", "Fenwood Road", "Mission Park", "Riverway", "Back of the Hill", "Heath Street"];


In [116]:
z_green = zeros((4,66));

for i = 1:66
    if green_stations[i] in Green1
        z_green[1,i] = 1
    end
    if green_stations[i] in Green2
        z_green[2,i] = 1
    end
    if green_stations[i] in Green3 
        z_green[3,i] = 1
    end
    if green_stations[i] in Green4 
        z_green[4,i] = 1
    end
end

In [117]:
modelGreen = Model(Gurobi.Optimizer)

@variable(modelGreen, x[1:directions, 1:times, 1:lines] >= 0)
@variable(modelGreen, u[1:directions, 1:times, 1:stations_g] >= 0)

@constraint(modelGreen, [d=1:directions, j=1:times, i=1:stations_g], 
    u[d,j,i] + (sum(capacity_green * x[d,j,l] * z_red[l,i] for l=1:lines)) >= AvgFlowGreen[i,j,d])

@objective(modelGreen, Min, sum( sum(cost_green * x[d,j,l] for l=1:lines ) + 
                sum(q * u[d,j,i] for i=1:stations_g) for d=1:directions,  j=1:times))

optimize!(modelGreen)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-04
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1188 rows, 1260 columns and 1548 nonzeros
Model fingerprint: 0xb3e5103e
Coefficient statistics:
  Matrix range     [1e+00, 9e+02]
  Objective range  [2e+00, 8e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 930 rows and 966 columns
Presolve time: 0.00s
Presolved: 258 rows, 294 columns, 570 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0166232e+06   8.041469e+03   0.000000e+00      0s
      90    1.5118911e+06   0.000000e+00   0.000000e+00      0s

Solved in 90 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.511891111e+06

User-callback calls 139, time in user-callback 0.00 sec
